In [1]:
import json
import pandas as pd
import numpy as np
import json

In [2]:
with open("../PubMed_Search/search_results/search_results_hanimals.txt", "r") as f:
    a = f.read()
    spec_papers_dic = json.loads(a)
# 加载判断了的病原菌信息
prediction = json.load(open("output/output_pat/total_predictions.json", 'r'))
result = {}
for i in prediction.keys():
    paper_id = i.split(":")[0]
    spec = i.split(":")[1]
    ncbi = "https://pubmed.ncbi.nlm.nih.gov/%s/" % paper_id
    result[i] = [paper_id, spec, prediction[i][1][0], ncbi]
result = pd.DataFrame(result, index = ["ID", "SPECIES", "rate", "link"]).T
prediction_extend = {}

for spec_type in spec_papers_dic:

    prediction_extend[spec_type] = {}
    
    for k, v in prediction.items():

        spec = k.split(":")[1]
        paper_id = k.split(":")[0]
        
        if spec in spec_papers_dic[spec_type]:

            if v[1][0] > 0.5:

                if spec not in prediction_extend[spec_type]:
                
                    prediction_extend[spec_type][spec] = {}
                    prediction_extend[spec_type][spec][paper_id] = spec_papers_dic[spec_type][spec][paper_id]
                    prediction_extend[spec_type][spec][paper_id]["predition"] = v
                else:
                    prediction_extend[spec_type][spec][paper_id] = spec_papers_dic[spec_type][spec][paper_id]
                    prediction_extend[spec_type][spec][paper_id]["predition"] = v

with open("output/output_pat/prediction_extend.json", "w") as f:
    json.dump(prediction_extend, f)

In [6]:
# 根据上述筛选出的样本的数据集
from mkdir import mkdir
col_ll = [
    ["样本量","num",{"1":"How many patients are ivolved in this study?"}],
    ["患者性别","gender",{"1":"What's gender of the patient?"}],
    ["年龄","age",{"1":"How old is patient?"}],
    ["证据等级","evi",{"1":"Where is pathogen isolated?"}],
    ["免疫状态","immu",{"1":"What's patient immunity state?"}],
#     ["预后","outcome",{"1":"What's patient outcome?"}]
]
for col_l in col_ll:
    
    col_name = col_l[0]
    col_ab = col_l[1]
    que_d = col_l[2]
    
    test_qa = []
    test_set = {"data": [{'paragraphs': [], 'title':"LungB"}], "version": "LungB"}

    for spec_type in spec_papers_dic:
        
        for pre_keys in prediction:
            
            if pre_keys.split(":")[1] in spec_papers_dic[spec_type]:
        
                qa = {'qas': [{
                    'id': pre_keys,
                    'question': que_d["1"].replace("pathogen", pre_keys),
                    "answers": []}],
                    'context': spec_papers_dic[spec_type][pre_keys.split(":")[1]][pre_keys.split(":")[0]]["texts"]["Ab"]+" "+
                    " ".join(sum(spec_papers_dic[spec_type][pre_keys.split(":")[1]][pre_keys.split(":")[0]]["Meshhead"],[])) } # 添加mesh词汇

                if len(spec_papers_dic[spec_type][pre_keys.split(":")[1]][pre_keys.split(":")[0]]["texts"]["Ab"]) > 0:
                    test_qa.append(qa)
   
    test_set['data'][0]["paragraphs"] = test_qa
    mkdir("input/input_%s/" % (col_ab))
    json.dump(test_set, open("input/input_%s/test_set.json" % (col_ab), "w"))